# Notebook Info
The goal of this notebook is to webscrape the specified website for three main data sets:
- Generation output in kWh for every trading period between 2015 to 2020.
- Fleet information of every hydrostation in the South Island.
- Network supply points data set to identify the location of each hydrostation.

Once scraped, the data is wrangled into the desired format.

# Code

## Packages

In [1]:
library(tidyverse)
library(xml2)
library(rvest)
library(httr)
#install.packages("readxl")
library(readxl)
#install.packages("furrr")
library(furrr)
future::plan(multicore)

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding


Loading required package: future



## Information variables
These variables hold the website url to be webscraped. The sub urls (power_url, stations_url, points_url) will be concatenated independently onto the main_url to form a single address.

In [2]:
# Main site address
main_url <- "https://www.emi.ea.govt.nz"
# Sub address for electricity generation information
power_url <- "/Wholesale/Datasets/Generation/Generation_MD"
# Sub address for hydrostations fleet
stations_url <- "/Wholesale/Datasets/Generation/Generation_fleet/Existing"
# Sub address for network supply points
points_url <- "/Wholesale/Reports/R_NSPL_DR?_si=v|3"

## Retrieve generation output data
The main website holds a dataset for every month of each year containing the generation output for every trading period, which occurs every 30 minutes beginning at 12:00 am.
These datasets are retrieved and filtered such that only the data sets within 2015 and 2020 are included.

In [3]:
## __Code_Explanantion__
# 1. The sub address for the page containing generation output is concatenated with the main url to produce
#    a single address. This address is passed into read_html() to acquire the source code.
# 2. The source code is filtered to identify the csv urls for each dataset containing the data.
# 3. The acquired urls are filtered by removing duplicates and urls that do not contain power_url sub address
#    within the string.
# 4.0. The character position for the first integer of the year within the urls is identified.
# 4.1. The year from each url is extracted.
# 5. A vector is produced containing the required years.
# 6. The urls are further filtered by removing years that are not within the required range.
# 7. The urls containing the datasets for generation are concatenated with the main url, ready for retrieval from
#    the website.
# 8. A single tibble is produced with the retrieved data sets from the urls. future_map_dfr is used from the 'furrr'
#    package as it allows parallelization, unlike map_dfr from the 'purrr' package. This was found to dramatically
#    increase the speed in which the datasets were retrieved.
# 9. The non hydro stations from the tibble are removed.

# Acquire html code from sub address
source <- paste(main_url, power_url, sep="") %>% read_html()  # 1.
"Source checkpoint"
# Identify all links in the given html class
all_links <- source %>% html_nodes(".table") %>% html_elements("a") %>% html_attr("href")  # 2.
"Links identification checkpoint"
# Identify appropriate urls within links and remove duplicates
filtered_links <- all_links[startsWith(all_links, power_url) & !duplicated(all_links)]  # 3.
"Links filtered checkpoint"
# Extract date information from filtered links
start_pos <- nchar(power_url) + 2  # 4.0.
dates <- substring(filtered_links , start_pos, start_pos + 3) # 4.1.
"Date extraction checkpoint"
# Set dates wanted
dates_wanted <- seq(from=2015,to=2020)  # 5.
# Select links containing csv data
selected_links <- filtered_links[dates %in% dates_wanted]  # 6.
"Links selected checkpoint"
# Extract csv data into dataframes
link <- paste(main_url, selected_links, sep="")  # 7.
generation_md <- link %>% future_map_dfr(read_csv, show_col_types = FALSE)  # 8.
"Generation retrieval checkpoint"

# Filter out non hydro stations
generation_md_hydro <- generation_md %>% filter(Tech_Code == "Hydro")  # 9.
"Data filter checkpoint"
"RETRIEVAL COMPLETE"
generation_md_hydro

## __Outcome__
# The final dataframe contains information of the generation output of most hydrostations in the South Island.
# This data is arranged in a long format.

[1] "Source checkpoint"

[1] "Links identification checkpoint"

[1] "Links filtered checkpoint"

[1] "Date extraction checkpoint"

[1] "Links selected checkpoint"

[1] "Generation retrieval checkpoint"

[1] "Data filter checkpoint"

[1] "RETRIEVAL COMPLETE"

Site_Code POC_Code Nwk_Code Gen_Code  Fuel_Code Tech_Code Trading_date
1     ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-01  
2     ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-02  
3     ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-03  
4     ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-04  
5     ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-05  
6     ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-06  
7     ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-07  
8     ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-08  
9     ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-09  
10    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-10  
11    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-11  
12    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-12  
13    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-13  
14    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-14  
15    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-15  
16    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-16  
17    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-17  
18    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-18  
19    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-19  
20    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-20  
21    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-21  
22    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-22  
23    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-23  
24    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-24  
25    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-25  
26    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-26  
27    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-27  
28    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-28  
29    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-29  
30    ARA       ARA2201  MRPL     aratiatia Hydro     Hydro     2020-12-30  
⋮     ⋮         ⋮        ⋮        ⋮         ⋮         ⋮         ⋮           
84081 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-02  
84082 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-03  
84083 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-04  
84084 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-05  
84085 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-06  
84086 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-07  
84087 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-08  
84088 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-09  
84089 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-10  
84090 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-11  
84091 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-12  
84092 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-13  
84093 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-14  
84094 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-15  
84095 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-16  
84096 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-17  
84097 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-18  
84098 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-19  
84099 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-20  
84100 WPI       WPG0331  TRPG     waipori   Hydro     Hydro     2015-01-21  
84

## Retrieve hydrostations fleet information
Acquire fleet information, including station name, general location and owner name.

In [4]:
## __Code_Explanation
# 1. The sub address for the page containing fleet information is concatenated with the main url to produce
#    a single address. This address is passed into read_html() to acquire the source code.
# 2. The source code is filtered to identify all links within the page.
# 3. The acquired urls are filtered by removing duplicates and urls that do not contain stations_url sub address
#    within the string.
# 4. The remaining url contains the sub address for the spreadsheet with the data required.
# 5. The sub address for the spreadsheet is concatenated with the main url to form a single address.
# 6. The spreadsheet is retrieved and saved as a temporary file.
# 7. The relevant sheet is selected to be read from the spreadsheet, and is filtered to remove non-hydro stations.
# 8. Only hydro stations in the South Island are selected and grouped by company assigned group names.
# 9. The irrelevant columns are removed, including comments and irrelevant data.


# Acquire html code from sub address
source <- paste(main_url, stations_url, sep="") %>% read_html()  # 1.
"Source Checkpoint"
# Identify all links in the given html class
all_links <- source %>% html_nodes(".xls") %>% html_elements("a") %>% html_attr("href")  # 2.
"Links identification checkpoint"
# Identify appropriate urls within links and remove duplicates
filtered_links <- all_links[startsWith(all_links, stations_url) & !duplicated(all_links)]  # 3.
selected_link <- filtered_links  # 4.
"Links filtered checkpoint"
# Save spreadsheet into a temporary file and the data into a tibble
url <- paste(main_url, selected_link,sep="")  # 5.
output <- GET(url, write_disk(spreadsheet <- tempfile(fileext = ".xls")))  # 6.
"Spreadsheet retrieval checkpoint"
generation_fleet <- read_excel(spreadsheet, sheet = "Generating Stations") %>% 
                        filter(Generation_Type == "Hydro")  # 7.
"Data extraction checkpoint"
generation_fleet_SI <- generation_fleet %>% 
                        filter(Island_Name == "SI - South Island") %>% 
                        group_by(GroupName) %>% 
                        arrange(.by_group = TRUE) # 8.
"Data South Island filtered checkpoint"
generation_fleet_SI <- generation_fleet_SI[, c('Station_Name', 
                                               'GroupName', 
                                               'Owner_Name', 
                                               'Operators_Name', 
                                               'Node_Name', 
                                               'Region_Name', 
                                               'Island_Name')] # 9.
"Data full filtering checkpoint"
"RETRIEVAL COMPLETE"
generation_fleet_SI

## __Outcome__
# The result is a dataframe containing naming and general location information for most hydro stations in the 
#  South Island.

[1] "Source Checkpoint"

[1] "Links identification checkpoint"

[1] "Links filtered checkpoint"

[1] "Spreadsheet retrieval checkpoint"

[1] "Data extraction checkpoint"

[1] "Data South Island filtered checkpoint"

[1] "Data full filtering checkpoint"

[1] "RETRIEVAL COMPLETE"

Station_Name           GroupName                   Owner_Name           
1  Argyle                 Branch Hydro Scheme         Trustpower           
2  Wairau                 Branch Hydro Scheme         Trustpower           
3  Clyde                  Clutha Hydro Scheme         Contact Energy       
4  Roxburgh               Clutha Hydro Scheme         Contact Energy       
5  Dillmans               Dillmans Hydro Power Scheme Trustpower           
6  Duffers                Dillmans Hydro Power Scheme Trustpower           
7  Kumara                 Dillmans Hydro Power Scheme Trustpower           
8  Kaniere Forks          Kaniere River               Trustpower           
9  McKays Creek           Kaniere River               Trustpower           
10 Paerau                 Paerau Gorge Power Scheme   Trustpower           
11 Highbank               Rangitata Diversion Race    Trustpower           
12 Montalto               Rangitata Diversion Race    Trustpower           
13 Horseshoe Bend         Teviot Hydro Scheme         Pioneer Generation   
14 Kowhai                 Teviot Hydro Scheme         Pioneer Generation   
15 Teviot                 Teviot Hydro Scheme         Pioneer Generation   
16 Aviemore               Waitaki Hydro Scheme        Meridian Energy      
17 Benmore                Waitaki Hydro Scheme        Meridian Energy      
18 Ohau A                 Waitaki Hydro Scheme        Meridian Energy      
19 Ohau B                 Waitaki Hydro Scheme        Meridian Energy      
20 Ohau C                 Waitaki Hydro Scheme        Meridian Energy      
21 Tekapo A               Waitaki Hydro Scheme        Genesis Energy       
22 Tekapo B               Waitaki Hydro Scheme        Genesis Energy       
23 Waitaki                Waitaki Hydro Scheme        Meridian Energy      
24 Amethyst               NA                          Westpower            
25 Arnold                 NA                          Trustpower           
26 Brooklyn Power Station NA                          Lloyd Wensley        
27 Cleardale              NA                          MainPower            
28 Cobb                   NA                          Trustpower           
29 Coleridge              NA                          Trustpower           
30 Deep Stream            NA                          Trustpower           
31 Falls Dam              NA                          Pioneer Generation   
32 Fox                    NA                          Trustpower           
33 Fraser                 NA                          Pioneer Generation   
34 Manapouri              NA                          Meridian Energy      
35 Mataura                NA                          Niblick Trust        
36 Monowai                NA                          Pioneer Generation   
37 Ngahere                NA                          Birchfield Minerals  
38 Onekaka                NA                          Onekaka Energy       
39 Opuha                  NA                          Alpine Energy        
40 Oxburn/Glenorchy       NA                          Pioneer Generation   
41 Pupu Hydro             NA                          Pupu Hydro Society   
42 Roaring Meg            NA                          Pioneer Generation   
43 Rochfort               NA                          Kawatiri Energy      
44 Talla Burn             NA                          Talla Burn Generation
45 Wahapo (Okarito Forks) NA                          Trustpower           
46 Waihopai               NA                          Trustpower           
47 Waipori                NA                          Trustpower           
48 Wye Creek              NA                          Pioneer Generation   
   Operators_Name        Node_Name Region_Name               Island_Name      
1  Trustpower            ARG1101   NEL - Nelson/Marlbourough SI - South Island
2  Trustpower            ARG1101   NEL - Nelson/Marlbourough SI - South Island
3  Contact Energy        CYD2201   OTG - O

## Retrieve Network supply points
Retrieve the data set containing information, specifically location, for each supply node in the network.

In [5]:
## __Code_Explanation__
# 1. The sub address for the page containing the network supply points is concatenated with the main url to produce
#    a single address. This address is passed into read_html() to acquire the source code.
# 2. The source code is filtered to identify all links within the page.
# 3. The acquired urls are filtered by removing duplicates and urls that do not start with the /Wholesale prefix
#    within the string. NA are also removed.
# 4. The remaining link contains the address for the network supply points.
# 5. The link is concatenated with the main url to form a single address.
# 6. The network supply point csv is retrieved and filtered, selecting location and identity information.
#    The identification column is renamed to  Node name to be used as a key for merging.

source <- paste(main_url, points_url, sep="") %>% read_html()  # 1.
"Source Checkpoint"
all_links <- source %>% html_nodes(".emi-btn-no-border") %>% html_elements("a") %>% html_attr("href")  # 2.
"Links Identification Checkpoint"
filtered_links <- all_links[startsWith(all_links, "/Wholesale") & !duplicated(all_links)] %>% na.omit()  # 3.
selected_link <- filtered_links  # 4.
"Links Filtered Checkpoint"
url <- paste(main_url, selected_link, sep="")  # 5.
supply_points <- url %>% read.csv(skip=6) %>% 
                            select(POC.code, NZTM.easting, NZTM.northing) %>% 
                            rename(Node_Name = POC.code)  # 6.
"Dataset Retrieval Checkpoint"
"RETRIEVAL COMPLETE"
supply_points

## __Outcome__
# The resulting data frame contains location information for each node in the supply network.

[1] "Source Checkpoint"

[1] "Links Identification Checkpoint"

[1] "Links Filtered Checkpoint"

[1] "Dataset Retrieval Checkpoint"

[1] "RETRIEVAL COMPLETE"

Node_Name NZTM.easting NZTM.northing
1    ABY0111   1424397      5097843      
2    AKK0011   1673612      6082717      
3    AKL0331   1758670      5903100      
4    ALB0331   1750929      5932699      
5    ALB1101   1750929      5932699      
6    ANI0331   1931640      5754616      
7    ANI0331   1931640      5754616      
8    APC0011   1763935      5912460      
9    APS0111   1483256      5243850      
10   ARA2201   1873657      5721161      
11   ARG1101   1616837      5386748      
12   ARI1101   1831847      5782925      
13   ARI1102   1831847      5782925      
14   ASB0661   1503871      5133909      
15   ASC0011   1564200      5182350      
16   ASY0111   1569375      5209968      
17   ASY0111   1569375      5209968      
18   ATI0111   1863993      5746266      
19   ATI0111   1863993      5746266      
20   ATI0112   1863993      5746266      
21   ATI0112   1863993      5746266      
22   ATI2201   1863993      5746266      
23   ATU1101   1479770      5314014      
24   AVI2201   1390245      5051586      
25   BAL0331   1350994      4875583      
26   BCK0011   1748190      5926820      
27   BCK0012   1748190      5926820      
28   BDE0111   1279333      4872602      
29   BDE0111   1279333      4872602      
30   BEN2202   1377232      5061385      
⋮    ⋮         ⋮            ⋮            
2045 WSC0011   1750950      5932751      
2046 WSC0011   1750950      5932751      
2047 WSC0011   1750950      5932751      
2048 WSL0111   1754500      5916700      
2049 WSL0111   1754500      5916700      
2050 WTK0111   1396274      5048061      
2051 WTK0331   1396274      5048061      
2052 WTK0331   1396274      5048061      
2053 WTK0331   1396274      5048061      
2054 WTK0331   1396274      5048061      
2055 WTK0331   1396274      5048061      
2056 WTK0331   1396274      5048061      
2057 WTK0331   1396274      5048061      
2058 WTK0331   1396274      5048061      
2059 WTK0331   1396274      5048061      
2060 WTN0111   1237336      4879291      
2061 WTN0661   1237336      4879291      
2062 WTS0011   1761811      5914199      
2063 WTS0011   1761811      5914199      
2064 WTS0011   1761811      5914199      
2065 WTU0331   1933253      5608737      
2066 WVY0111   1740108      5598255      
2067 WWC0011   1747953      5916477      
2068 WWC0011   1747953      5916477      
2069 WWC0011   1747953      5916477      
2070 WWC0011   1747953      5916477      
2071 WWD1102   1740630      5429932      
2072 WWD1103   1740630      5429932      
2073 WWK0111   1866750      5708080      
2074 WWK0111   1866750      5708080

## Add NZTM coordinated from supply points to generation fleet
The location and identification dataframe is merged with the generation fleet dataframe.

In [6]:
## __Code_explanation__
# 1. supply_points and generation_fleet_SI are merge by using 'Node_Name' as the key.
# 2. Duplicate station names are removed.

gen_fleet <- merge(generation_fleet_SI, supply_points, by='Node_Name')  # 1.
gen_fleet <- gen_fleet[!duplicated(gen_fleet$Station_Name),]  # 2.
# Gen fleet contains hydrstation fleet with their locations, ot including one hydrostation without a location.
gen_fleet

## __Outcome__
# The resulting data frame contains more detailed location information of each hydrostation, ready to be correlated to
#  the river locations.

Node_Name Station_Name           GroupName                  
1   ABY0111   Opuha                  NA                         
4   ARA2201   Amethyst               NA                         
6   ARG1101   Argyle                 Branch Hydro Scheme        
9   ARG1101   Wairau                 Branch Hydro Scheme        
12  ASB0331   Montalto               Rangitata Diversion Race   
14  ASB0661   Highbank               Rangitata Diversion Race   
32  ASB0661   Cleardale              NA                         
50  AVI2201   Aviemore               Waitaki Hydro Scheme       
52  BEN2202   Benmore                Waitaki Hydro Scheme       
53  BLN0331   Waihopai               NA                         
55  BWK1101   Waipori                NA                         
58  CML0331   Roaring Meg            NA                         
60  COB0661   Cobb                   NA                         
62  COL0661   Coleridge              NA                         
65  CYD0331   Horseshoe Bend         Teviot Hydro Scheme        
69  CYD0331   Teviot                 Teviot Hydro Scheme        
73  CYD0331   Fraser                 NA                         
77  CYD0331   Kowhai                 Teviot Hydro Scheme        
81  CYD0331   Talla Burn             NA                         
85  CYD2201   Clyde                  Clutha Hydro Scheme        
87  DOB0331   Arnold                 NA                         
89  DOB0331   Ngahere                NA                         
91  FKN0331   Oxburn/Glenorchy       NA                         
94  FKN0331   Wye Creek              NA                         
97  GOR0331   Mataura                NA                         
100 HKK0661   Kaniere Forks          Kaniere River              
104 HKK0661   McKays Creek           Kaniere River              
108 HKK0661   Fox                    NA                         
112 HKK0661   Wahapo (Okarito Forks) NA                         
116 HWB0331   Deep Stream            NA                         
137 KUM0661   Dillmans               Dillmans Hydro Power Scheme
139 KUM0661   Kumara                 Dillmans Hydro Power Scheme
141 MAN2201   Manapouri              NA                         
143 MOT0111   Brooklyn Power Station NA                         
145 MPI0661   Onekaka                NA                         
147 MPI0661   Pupu Hydro             NA                         
149 NMA0331   Monowai                NA                         
151 NSY0331   Paerau                 Paerau Gorge Power Scheme  
153 NSY0331   Falls Dam              NA                         
155 OHA2201   Ohau A                 Waitaki Hydro Scheme       
157 OHB2201   Ohau B                 Waitaki Hydro Scheme       
159 OHC2201   Ohau C                 Waitaki Hydro Scheme       
161 ORO1101   Rochfort               NA                         
189 ROX1101   Roxburgh               Clutha Hydro Scheme        
191 TKA0111   Tekapo A               Waitaki Hydro Scheme       
194 TKB2201   Tekapo B               Waitaki Hydro Scheme       
197 WTK0111   Waitaki                Waitaki Hydro Scheme       
    Owner_Name            Operators_Name        Region_Name              
1   Alpine Energy         Contact Energy        SCN - South Canterbury   
4   Westpower             Westpower             WEC - West Coast         
6   Trustpower            Trustpower            NEL - Nelson/Marlbourough
9   Trustpower            Trustpower            NEL - Nelson/Marlbourough
12  Trustpower            Trustpower            CAN - Canterbury         
14  Trustpower            Trustpower            CAN - Canterbury         
32  MainPower             MainPower             CAN - Canterbury         
50  Meridian Energy       Meridian Energy       SCN - South Canterbury   
52  Meridian Energy       Meridian Energy       SCN - South Canterbury   
53  Trustpower            Trustpower            NEL - Nelson/Marlbourough
55  Trustpower            Trustpower            OTG - Otago/Southland 